# Alice, Bob & Eve: Digital signatures using quantum-safe ML-DSA

The algorithms mentioned in previous parts (RSA, ECDSA) are not quantum-safe. Once Eva will have quantum computer available, she can forge the signatures. Luckily, we already have two post-quantum digital signature algorithms available. Here we'll take a look at the ML-DSA (Module‑Lattice‑based Digital Signature Algorithm), formerly known as CRYSTALS-Dilithium.

This algorithm is available natively in .NET 10. For older versions you can use the `Microsoft.Bcl.Cryptography` package. We will be actually doing this, because current version of .NET Interactive is using .NET 9.

## Hack: Installing the NuGet package 

In regular .NET 10 applications you do not need to install this NuGet package, as everything is part of the runtime itself. However, at time of writing, .NET Interactive (which is used in notebooks) uses version 9.0 and we have to import the package manually:

In [21]:
// Install the Microsoft.Bcl.Cryptography package 
#r "nuget: Microsoft.Bcl.Cryptography"

// Import namespace
using System.Security.Cryptography;

// Show what runtime we're actually on
Console.WriteLine($"Runtime version:  {System.Runtime.InteropServices.RuntimeInformation.FrameworkDescription}");

// Check whether ML-KEM is supported on this platform
Console.WriteLine($"ML-KEM supported: {MLKem.IsSupported}");

Installed Packages Microsoft.Bcl.Cryptography, 10.0.1

Runtime version:  .NET 9.0.11
ML-KEM supported: True


## Alice

As usual, Alice will generate her key pair. Notice that the keys are significantly longer not only than ECDSA, but also than RSA keys. But this is completely different scheme, so it's quite hard to compare them.

The API is kind of similar to the one exposed by `AsymmetricAlgorithm` base class, but the post-quantum algorithm classes are not derived from this class, because the philosophy and usage is different enough.

The ML-DSA is available in three variants: ML-DSA-44, ML-DSA-65 and ML-DSA-87. The numbers are arbitrary; they are sequential numbers of original CRYSTALS-Dilithium parameter sets and do not directly relate to any specific parameters, such as key sizes.

Higher number is "better" in terms of security (and "worse" in terms of performance and data size). The following table shows the three variants and their parameters (security roughly equivalent to AES with given key lenghs and size of various aspects):

| Variant | AES Equivalent Length | Public Key Length (bytes) | Public Key Length (bits) | Private Key Length (bytes) | Private Key Length (bits) | Signature Length (bytes) | Signature Length (bits) |
|--------|---------------:|--------------------:|-------------------:|---------------------:|--------------------:|-------------------:|------------------:|
| MLDSA44 | 128 | 1312 | 10496 | 2528 | 20224 | 2420 | 19360 |
| MLDSA65 | 192 | 1952 | 15616 | 4000 | 32000 | 3293 | 26344 |
| MLDSA87 | 256 | 2592 | 20736 | 4864 | 38912 | 4595 | 36760 |

For this demo we will be using the simplest variant, ML-DSA-44:

In [22]:
using System.Security.Cryptography;

var mldsa = MLDsa.GenerateKey(MLDsaAlgorithm.MLDsa44);
var privateKeyBytes = mldsa.ExportMLDsaPrivateKey();
var publicKeyBytes = mldsa.ExportMLDsaPublicKey();


// Display the keys
Console.WriteLine($"Alice's Private Key ({privateKeyBytes.Length * 8} bits):");
Console.WriteLine(Convert.ToBase64String(privateKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Private Key (20480 bits):
6fuUTOlO3HjRH5DFoAYnx2Xg9OzRFTn1unD5KQ8jjRIQGVtU+M1LUZ5aYSujG1bjZOiS1Nchbuhx
GJ9VNumW80FyYEgt2iywwMz+oreshPasTtizbqvl5PD5X7AGE+Dv7RE6zS7UGQFX6he4tcRWPxX9
9CCer/oAbXdj51Qt0E0ImCViJAmbgJCACC4bsTFixA0EEUUIA0wQgkkJEUIQGAAUA0kLIVCRsjBg
IIwEoYUioDCIsGjLIAYhSClUNI0ZkWURF0jiliUkMkwjByzEoCUjhi2UtFDLSIngEAUKGE6EKBEb
xlFLBo5Aok2AuCGhRmjUkIkCJHBCwgARuYwhgHAiIYwbI4RiMAIJJ0QIOGQhpCXJKE4YBCLItmQg
sUwRhRDMJGyaCCqaBEpCtpEconEkFi3axglSBo6DRlCgthBRQgwICGDkFoGasG0EtYAkkyyEQA7I
MAjgMgzMgHFDgi1jIiXTIkliMkoDCWAMgwiRJhBCEglaMIpghgxaRkaaNAnbCATZAgrMlm0bNwwg
qS1JBEkBIyiLKBAkxyhRBoxYMGEahxHEyAiTAkYZRwogMVDYtgwipSEMoIGgCE4KMm5SpAQZFjAA
okkMs4RRIkLjpGRTFEFZQJECoXFKIgbbIIkBxxARBhCCQHALJ2ZIiEjAIilaQoIkKSwjJIpAwi0S
JjHbNECkIiVYKBASFmEQIzCSsC2SFEXTIEwYREpLsoyBIgWioHHkpEhbiBGEMCQbBEAKNmEKMwAT
NAhZRkXJFISAkolkBo6MsoRDBojTpFHRtISMCIUkCYkCskxZlmEQQoAEI1FjNkAIQ4WZGGyYxgGa
GJHEMgDDME4QhjHggI2TppEQg2XJMokhFASBhIgEo4jhsHGLRGLBBAQMJ2TclIGLRo3UtGEgOIxa
QgqCloUARFLMOAAMBwHJCAhCQCkcqC3JRjHZFobKBo

In [23]:
Console.WriteLine($"Alice's Public Key ({publicKeyBytes.Length * 8} bits):");
Console.WriteLine(Convert.ToBase64String(publicKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Public Key (10496 bits):
6fuUTOlO3HjRH5DFoAYnx2Xg9OzRFTn1unD5KQ8jjRIuh0FibHq3IIzJk9+DT9Z4HTtY5mAE9OE6
bo0IbhWnW4JliTcQ6DGDjS3OyefkJmEaZ7i+hGTkzbU9Nyh6DZWSR09MPEC5cAcrFEPGPDp7bghR
rTGwE3DJCGXxHRgSVvnaFxgFusd4VJ31zoz8oO+7BlejV3COqpgGV4/0a8cyPJ1Lbfr4fJckQURy
lBQvHAsv8KITa6l2uJZQ0PYpWW4iKygVWUcX4MhXuOM6IfrRz+JO+DukXedxgtTfTLztVavRM/Wg
sFNdpOMiGBUVZS3jcGYGBPBkLTIRnN+57uZfN7wtJ5/NPDZwKKKyTnm68UqMwHUBwQvUDogsesCI
qOSFrvzEOzwgvkTHJ9SO+2Ipik8/kknOqvG6OzNg8BfTw98N8uv0699+AJeB33lS2GQyuviCowuw
FENnkKfmM/2mjuhMzn85vIkVpQgO5CwWUWX66Ksz3958mlDuUOV3Y3TiF2q9EEJO+OR1iVSg98ML
ABcxggrAULEMiotyiPJTiL3BR5cS0DhOxPDdbJy3O8Cc+9m5doaCjE2IRwJ8H0FA9MkRpDO36+6M
i2MY6KyM15maRo08Dj3SuvuyH9mi91avGdlUlFLbJGEs6825vJxzDqKexhkXaJzR+BbHCAlwIjoP
ko5jcIm0XVIprbwpo1fs6fvNYuxtFF6Zmt+/A0jklTm6sRLBORrEVbfKnsq8G6RL1JNvsATGFhxD
Gz9UPKzE0xRyq/uiq6IO3IstRnec/tnBTs3qL8TLZC03XafMSO4AS67llZMsU5bTTooAkRjHDdkC
BXS/Ztbxs7kTQ4jNoJG0nwoCuuE+l3VUZj+q6JLeLD+yTYlcEhhkIuWkcl5029vvkXK+ZW4I1Oof
2AP/yT379gjCXQEO/PLMQsKDj+uAbljB+8+r1hukIuI

Alice will then generate signature of her message. Notice that the signature itself is also larger than DSA or RSA:

In [24]:
var messageText = "Bob, I love you!";
var messageBytes = System.Text.Encoding.UTF8.GetBytes(messageText);
Console.WriteLine($"Original Message: {messageText}");
Console.WriteLine();

// Sign the message
var signatureBytes = mldsa.SignData(messageBytes);
var signatureText = Convert.ToBase64String(signatureBytes, Base64FormattingOptions.InsertLineBreaks);
Console.WriteLine($"Signature:");
Console.WriteLine(signatureText);

Original Message: Bob, I love you!

Signature:
cXYrOmtm8nTvsxzHndCYrRCXYWP8bydljuEUXdbv+N/gfqslSudAXTCXaJyuIhcxyX/pJ3iU0Y6b
87W/k+oM99MvObC2WuS2gVLjrC47Hb57TKF80QF1lOpsE0dcexm2vovEZdjfEF7+H+0W8hy37CJ7
u35cxTQNvgCRNxc1BMlzenP5eiymlzku+dC/1kW6uhvBm4FfZzqepWnSXbMi/w8ck+FLVtgwwr9P
9/bbruPeSjvrE+QhfpttBSappAXJvQYJYUlhJeMYNkF+Nbe1bv81N+8/M99G1T1uLu2dx8JkBY8/
4XJ6+plYcSIMcdxhXdL81/4mlS2br9/g90D24bv7/N9yxzdmbDIeiUjVhuq9uZaQAiPGTpcrur2H
r1kyc3hTsx0EHuwnffaeZW83PQTK5a7GKX5O7M/6LQLnjLsYKzcwRFutGWcUBNMRDW1/5TnS8qgS
ZTOzaT2dd1tOHf9nO4qbhOQbfTMdPTxf40dgvMsJofnLuoXrvPhllWOH5nljFjk7gLBpfhZyEJgg
uE6EorIDQ1nDHQ3OjFLhuqwNUqB0V5GPsCjpiTaGQ726cfWE0HcEdjjTizFezrrrqAqyrk/Gf+hM
wqLQvy1DZiXeuCazk5cdRGNDW8t3B6H0zoSCl5ciBmlFqXT/Luk2INhfeY5Oh02JYyafvaDodNkV
UdTVyHd5DfH8+HeA4fH3qF6FBof/RyHF60L62KSKZsKbsTa9c7/0J+C3yeyb94E+KUpW0160dDot
uOONu/ZqSkviaQwVm2dC9fIkknPpICEj+jkAAyt1yuuMhVgU3NEcF/qwiDleruKRZF+PPWbAn3z4
9dZYywlOJM9zuuZkFQa4bO8vr02x1xNNrlgWHKbXFDwlVkNlSCeOpf+KZMysUx78OXC9Vy/oOl9l
9arabJejK2MpVky2Dhh+KwH0cnnwX

## Bob

Bob will verify the signature in a fashion similar to RSA or DSA:

In [25]:
// Function to verify signature
bool VerifySignature(byte[] publicKey, string message, byte[] signature) {
    // Create MLDsa instance for verification using the public key
    using var mldsaVerify = MLDsa.ImportMLDsaPublicKey(MLDsaAlgorithm.MLDsa44, publicKey);

    // Convert message to bytes
    var messageBytesVerify = System.Text.Encoding.UTF8.GetBytes(message);

    // Verify the signature
    return mldsaVerify.VerifyData(messageBytesVerify, signature);
}

Console.WriteLine($"Is 'Bob, I love you!' valid? {VerifySignature(publicKeyBytes, "Bob, I love you!", signatureBytes)}");
Console.WriteLine($"Is 'Bob, I hate you!' valid? {VerifySignature(publicKeyBytes, "Bob, I hate you!", signatureBytes)}");

Is 'Bob, I love you!' valid? True
Is 'Bob, I hate you!' valid? False
